In [31]:
import arcpy, os, numpy
from arcpy import env

In [3]:
env.workspace = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"
env.overwriteOutput = True

In [4]:
arcpy.management.MakeFeatureLayer("parcels_FeatureToPoint", "baseyear_hh", "ohh > 0")

<Result 'baseyear_hh'>

In [5]:
fieldList = arcpy.ListFields("baseyear_hh")

In [6]:
field_names = [f.name for f in fieldList]

In [7]:
field_names

['OBJECTID',
 'Shape',
 'btype',
 'nrsqft',
 'rsqft',
 'du',
 'yrbuilt',
 'lpid',
 'pundev',
 'dev_land',
 'developed',
 'obtype',
 'orsqft',
 'onrsqft',
 'odu',
 'zid',
 'rezoned',
 'city',
 'annexed',
 'ozid',
 'AreaPerJob',
 'isNonRes',
 'jobs',
 'ojobs',
 'hh',
 'ohh',
 'ORIG_FID']

In [8]:
newfield = "sa_jobs"

In [11]:
desc = arcpy.Describe("baseyear_hh")

In [12]:
fields = desc.fields

In [20]:
arcpy.ListFields("baseyear_hh", "jobs")[0].type

'Double'

In [9]:
if newfield in field_names:
    pass
else:
    arcpy.AddField_management("baseyear_hh", newfield, "FLOAT", "", "", 20)

In [ ]:
with arcpy.da.UpdateCursor("baseyear_hh", ['ORIG_FID', 'sa_jobs']) as cursor: 
    for row in cursor:
        arcpy.management.SelectLayerByAttribute("baseyear_hh", "NEW_SELECTION", "ORIG_FID = {0}".format(row[0]), None)
        arcpy.management.SelectLayerByLocation("baseyear_hh", "COMPLETELY_WITHIN", "SABikingJobs", 
                                               None, "NEW_SELECTION", "NOT_INVERT")
        table = arcpy.AddJoin_management("SABikingJobs", "ObjectID", "baseyearJobs_FeatureToPoint", "ORIG_FID", "KEEP_COMMON")
        arcpy.CopyFeatures_management(table, "joinedtable")
        fieldsum = arcpy.da.TableToNumPyArray("joinedtable", "baseyearJobs_FeatureToPoint_jobs", skip_nulls=True)
        row[1] = fieldsum["baseyearJobs_FeatureToPoint_jobs"].sum()
        cursor.updateRow(row)